In [1]:
import numpy as np
import netket as nk
import functions as f

/home/keisuke/Documents/Research/sshfs/DimerMaster/functions.py:65: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [2]:
from conf import *
from numba import njit

In [3]:
np.zeros(3, dtype=np.intp)[0:3]
np.random.rand(4)
5.1 //3

1.0

In [4]:
@njit
def foo(x,y):
    print((x//y).astype(np.int64))
    return x

In [5]:
x = np.random.randint(0,100, 10)
y = np.random.randint(1,10, 10)

In [6]:
foo(x,y)

[12  6 14 29  8 18 11  2 45 13]


array([24, 44, 28, 89, 67, 56, 88, 12, 91, 95])

In [7]:
h = 1.0
V = 1.0

length = [4, 4]
name = 'h={}V={}l={}'.format(h, V, length)
n_chains = 10
sweep_size = 70
n_samples = 10000



g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = alpha, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)


sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition, length = length, n_chains=n_chains, sweep_size = 70)
sa.generate_samples(10) # discard the begginings of metropolis sampling.
print('discard samples')
samples_state = sa.generate_samples(int(n_samples / n_chains))
samples_state = samples_state.reshape(-1, ma.hilbert.size)

print('prepared initial samples')




4
discard samples
prepared initial samples


In [18]:
import numpy as _np
@njit
def _get_conn_flattened_kernel(
        x,
        sections,
        local_states,
        basis,
        constant,
        diag_mels,
        n_conns,
        all_mels,
        all_x_prime,
        acting_on,
        acting_size
    ):
    
    
    batch_size = x.shape[0]
    n_sites = x.shape[1]

    assert sections.shape[0] == batch_size

    n_operators = n_conns.shape[0]
    xs_n = _np.empty((batch_size, n_operators), dtype=_np.intp)

    max_conn = 0

#     acting_size = np.int8(acting_size)
#     acting_size = 4
    tot_conn = 0

    sections[:] = 1
    
    for i in range(n_operators):
        n_conns_i = n_conns[i]
        x_i = (x[:, acting_on[i]] + 1) / 2
        s = np.zeros(batch_size)
        for j in range(acting_size):
            s += x_i[:, j] * basis[j]
        xs_n[:, i] = s
        sections += n_conns_i[xs_n[:, i]]
    tot_conn=sections.sum()
    
    s = 0 
#     sec = sections.copy()
    for b in range(batch_size):
        s += sections[b]
        sections[b] = s



    x_prime = _np.empty((tot_conn, n_sites), dtype=_np.int8) # x.shpae[0] is number of connected elements of hamiltonian from batch of states. 
    mels = _np.empty(tot_conn, dtype=_np.complex128)

    
    c = 0
    for b in range(batch_size):
        c_diag = c
        mels[c_diag] = constant
        x_batch = x[b]
        xs_n_b = xs_n[b]
        x_prime[c_diag] = _np.copy(x_batch)
        c += 1
        for i in range(n_operators):

            # Diagonal part
            mels[c_diag] += diag_mels[i, xs_n_b[i]]
            n_conn_i = n_conns[i, xs_n_b[i]]

            if n_conn_i > 0:
                sites = acting_on[i]

                for cc in range(n_conn_i):
                    mels[c + cc] = all_mels[i, xs_n_b[i], cc]
                    x_prime_cc = x_prime[c + cc]
                    x_prime_cc[:] = _np.copy(x_batch)
                    x_prime_cc[sites] = all_x_prime[i, xs_n_b[i], cc]
#                     x_prime[c + cc] = x_prime_cc
#                     x_prime[c + cc] =  _np.copy(x_batch)
#                     for k in range(acting_size): 
#                         x_prime[c + cc, sites[k]] = all_x_prime[
#                             i, xs_n[b, i], cc, k
#                         ]
                c += n_conn_i


    return x_prime, mels

In [47]:
batch_size = samples_state.shape[0]
basis = op._basis[::-1].copy()
sections = np.zeros(batch_size, dtype=np.int64 )
sections1 = np.zeros(batch_size, dtype=np.int64 )

In [48]:
# %%timeit

# samples_state = np.random.choice([-1,1], (10000,32)).astype(np.int64)
x_prime, mels = _get_conn_flattened_kernel(
    samples_state,
    sections,
    op._local_states,
    basis,
    op._constant,
    op._diag_mels,
    op._n_conns,
    op._mels,
    op._x_prime,
    op._acting_on,
    op._acting_size[0]
)

In [49]:
# %timeit
x_prime1, mels1 = op.get_conn_flattened(samples_state, sections1)

In [50]:
sections == sections1

array([ True,  True,  True, ...,  True,  True,  True])

In [15]:
1000 * 15.6 

15600.0

In [13]:
15.6/10000

0.00156

In [11]:
x_prime[0]

array([-1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
      dtype=int8)

In [54]:
sections

array([    0,     6,    10, ..., 59882, 59889, 59896])

In [55]:
np.unique(sections[1:] - sections[:-1])

array([4, 5, 6, 7, 9])

In [19]:
sections

array([     0,     14,     35, ..., 166458, 166474, 166491])

In [36]:
sections

array([     0,     16,     27, ..., 165955, 165969, 165985])

In [37]:
%%timeit 
sections = np.zeros(batch_size, dtype=np.int64 )
# samples_state = np.random.choice([-1,1], (10000,32)).astype(np.int64)
x_prime, mels = op.get_conn_flattened(samples_state, sections)

55.8 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
sections

array([     0,     16,     27, ..., 165955, 165969, 165985])

In [212]:
(x_prime_ == x_prime).all()

True

In [146]:
x_prime.shape

(249070, 32)

In [ ]:
sections